<a href="https://colab.research.google.com/github/vaniamv/dataprocessing/blob/main/spark/examples/02-dataframes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/lucprosa/dataeng-basic-course/blob/main/spark/examples/02-dataframes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataframe
-

# Setting up PySpark

In [1]:
%pip install pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Spark Course').config('spark.ui.port', '4050').getOrCreate()
sc = spark.sparkContext

# DataFrames
- Untyped Datasets
- Similar to tables in relational databases
- DataFrames are just Dataset of Rows in Scala and Java API. These operations are also referred as “untyped transformations” in contrast to “typed transformations” come with strongly typed Scala/Java Datasets.

In [3]:
# Creating from RDDs
# .toDF()

lst = [("c1", "v1"), ("c2", "v2"), ("c3", "v3")]
rdd = sc.parallelize(lst)
df = rdd.toDF(["col", "value"])
df.show()



+---+-----+
|col|value|
+---+-----+
| c1|   v1|
| c2|   v2|
| c3|   v3|
+---+-----+



In [4]:
# Using "createDataFrame" + StructTypes

from pyspark.sql.types import *

data = [("c1", "v1"), ("c2", "v2"), ("c3", "v3")]

schema = StructType([
    StructField("col", StringType(), True),
    StructField("value", StringType(), True)
])

df = spark.createDataFrame(data, schema=schema)
df.show()


+---+-----+
|col|value|
+---+-----+
| c1|   v1|
| c2|   v2|
| c3|   v3|
+---+-----+



In [ ]:
# Read data from data sources
https://spark.apache.org/docs/3.5.2/sql-data-sources.html


In [6]:
!mkdir -p /content/files/samples/

In [7]:
# from csv
import csv

# creating csv file
with open('/content/files/samples/file.csv', 'w', newline='') as csvfile:
    fieldnames = ['col', 'value']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=";")
    writer.writeheader()
    writer.writerow({'col': 'c1', 'value': 'v1'})
    writer.writerow({'col': 'c2', 'value': 'v2'})
    writer.writerow({'col': 'c3', 'value': 'v3'})

# read csv file
df = spark.read.format("csv").load("/content/files/samples/file.csv", sep=";", header=True)
df.show()

+---+-----+
|col|value|
+---+-----+
| c1|   v1|
| c2|   v2|
| c3|   v3|
+---+-----+



In [8]:

# from json
json = """[{"col": "c1", "value": "v1"}, {"col": "c2", "value": "v2"}, {"col": "c3", "value": "v3"}]"""

text_file = open("/content/files/samples/file.json", "w")
text_file.write(json)
text_file.close()

# read from json
df = spark.read.json("/content/files/samples/file.json")
df.show()

+---+-----+
|col|value|
+---+-----+
| c1|   v1|
| c2|   v2|
| c3|   v3|
+---+-----+



In [9]:
# from parquet

# Generating parquet
lst = [("c1", "v1"), ("c2", "v2"), ("c3", "v3")]
rdd = sc.parallelize(lst)
df = rdd.toDF(["col", "value"])
df.write.format("parquet").mode("overwrite").save("/content/files/samples/parquet")


In [10]:
# read from parquet
df2 = spark.read.format("parquet").load("/content/files/samples/parquet")
df2.show()

+---+-----+
|col|value|
+---+-----+
| c1|   v1|
| c2|   v2|
| c3|   v3|
+---+-----+



In [11]:
# Check schema
df.printSchema()
print(df.schema)
print(df.columns)

root
 |-- col: string (nullable = true)
 |-- value: string (nullable = true)

StructType([StructField('col', StringType(), True), StructField('value', StringType(), True)])
['col', 'value']


In [12]:
# counting items in the dataframe
print(df.count())
print(df.take(5))
df.head(5)

3
[Row(col='c1', value='v1'), Row(col='c2', value='v2'), Row(col='c3', value='v3')]


[Row(col='c1', value='v1'),
 Row(col='c2', value='v2'),
 Row(col='c3', value='v3')]

In [13]:
# check explain plan
df.explain("cost")

== Optimized Logical Plan ==
LogicalRDD [col#83, value#84], false, Statistics(sizeInBytes=8.0 EiB)

== Physical Plan ==
*(1) Scan ExistingRDD[col#83,value#84]




In [18]:
df.toJSON().first()

'{"col":"c1","value":"v1"}'

In [19]:
df.toPandas()

,col,value
0,c1,v1
1,c2,v2
2,c3,v3


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('col').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('value').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('index', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('col')):
  _plot_series(series, series_name, i)
  fig.legend(title='col', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['value'].value_counts()
    for x_label, grp in _df_6.groupby('col')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('col')
_ = plt.ylabel('value')

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_7['col'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_7, x='index', y='col', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_8['value'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_8, x='index', y='value', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [22]:
# createOrReplaceTempView
df.createOrReplaceTempView("my_table")
spark.sql("select * from my_table").show()




+---+-----+
|col|value|
+---+-----+
| c1|   v1|
| c2|   v2|
| c3|   v3|
+---+-----+



In [23]:
df.unionAll(df).show()

+---+-----+
|col|value|
+---+-----+
| c1|   v1|
| c2|   v2|
| c3|   v3|
| c1|   v1|
| c2|   v2|
| c3|   v3|
+---+-----+



In [26]:
spark.table("my_table").show()

+---+-----+
|col|value|
+---+-----+
| c1|   v1|
| c2|   v2|
| c3|   v3|
+---+-----+



In [24]:
# createOrReplaceGlobalTempView
df.createOrReplaceGlobalTempView("my_table2")
spark.sql("select * from globaL_temp.my_table2").show()

+---+-----+
|col|value|
+---+-----+
| c1|   v1|
| c2|   v2|
| c3|   v3|
+---+-----+



## write operations

- df.write.format("parquet").save(path)
- df.write.format("json").save(path)
- df.write.format("csv").save(path)

### if using delta.io
- df.write.format("delta").saveAsTable(table_name)

